In [ ]:
# Požymių skaičiavimas EKG pūpsniams.

# Požymių skaičiavimas atliekamas su micro moduliu, kiekvienam ECG pūpsniui atskirai.
# Požymiai įrašomi į failą, csv formatu.

# Įrašai įmami iš sąrašo SubjCodes, kuris  paimamas iš mokymo,
# validavimo, testavimo sarašų, pvz. train_subjcode_lst.csv. Sąrašai suformuoti
# su skriptu 4_zive_creat_beats_attrib_split_v3.ipynb  Visiems įrašams iš šių
# sąrašų egzistuoja informacija apie pūpsnius faile all_beats_attr.
# 
# Šis variantas pritaikytas npy formato zive įrašams, kuriems pakeistas, lyginant su 
# originaliais įrašais, failo vardas iš `file_name` į `SubjCode`, pridedant `userNr`
# prie `file_name`. 
 
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import sys, os, json
from pathlib import Path

from zive_util_vu import create_dir, zive_read_df_rpeaks
from zive_util_vu import runtime, split_SubjCode, count_lines_enumrate
from zive_util_vu import get_userId, read_rec, get_filename 

# from zive_cnn_fda_vu_v1_micro import zive_read_file_1ch
# Pastaba: zive_read_file_1ch importuoju iš zive_cnn_fda_vu_v1, nors ji yra ir zive_util_vu.py
# tam, kad atskirti funkcijas, kurios importuojamos skripte zive analysis, nuo tų funkcijų,
# kurios reikalingos tik zive_accuracy_cnn_vu_v1 ir v2. 

print(tf.__version__)

# Modulis EKG signalo pūpsnių požymių skaičiavimui su užduotu modeliu

import keras, pickle
from get_beat_features_fda_vu import get_beat_features_fda_vu_v1

def get_beat_features_set_fda_vu_v1(signal, df_rpeaks, idx_lst):
# Apskaičiuojami užduotų EKG signalo pūpsnių (per idx_lst) požymiai ir iš jų
# suformuojamas požymių dataframe masyvas

    all_beats =  {'N':0, 'S':1, 'V':2, 'U':3, 'F':3}
    
    beat_features_set = pd.DataFrame()
    omit_idx_set = pd.DataFrame()

    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()
    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])
    
    # print("\nGet beat features set from signal:")
    for idx in idx_lst:
        beat_features, omit_idx = get_beat_features_fda_vu_v1(signal, atr_sample, idx)
        # beat_features_set = beat_features_set.append(beat_features, ignore_index=True)
        if omit_idx.empty:
            beat_features['label'] = test_labels[idx]
            beat_features_set = pd.concat([beat_features_set, beat_features])
        else:
            omit_idx_set = pd.concat([omit_idx_set, omit_idx])

    # Konvertuojame int pozymius į float64
    beat_features_set['RRl'] = beat_features_set['RRl'].astype(float)
    beat_features_set['RRr'] = beat_features_set['RRr'].astype(float)

    return beat_features_set, omit_idx_set

In [ ]:
# Pasiruošimas ir nurodomi bendri parametrai

print("Skriptas EKG įrašų pūpsnių požymių skaičiavimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    # Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, GUNDAS
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Bendras vietinės talpyklos aplankas
db_folder = 'DUOM_VU'

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy), anotacijomis (.json)
# ir failais su EKG įrašų SubjCode sąrašais 
rec_dir = Path(db_path, 'records_npy')

# Failai su EKG įrašų SubjCode sąrašais
train_subjcode_lst_fname = 'train_subjcode_lst.csv' 
validation_subjcode_lst_fname = 'validation_subjcode_lst.csv' 
test_subjcode_lst_fname = 'test_subjcode_lst.csv' 

# Pūpsnių atributų failas  - atsisakome
# all_beats_attr_fname = 'all_beats_attr_z.csv'
# Failai su požymių masyvais

# Nuoroda į aplanką su EKG požymiais
features_dir = Path(db_path, 'sets')

# Failai požymių masyvams
train_feature_set_fname = 'train_features_tst.csv' 
validation_feature_set_fname = 'validation_features_tst.csv' 
test_feature_set_fname = 'test_features_tst.csv' 

# Failai omitted masyvams
train_omitted_set_fname = 'train_omitted_tst.csv' 
validation_omitted_set_fname = 'validation_omitted_tst.csv' 
test_omitted_set_fname = 'test_omitted_tst.csv' 

# Masyvai pūpsnių klasių formavimui
# selected_beats = {'N':0, 'S':1, 'V':2}
# all_beats =  {'N':0, 'S':1, 'V':2, 'U':3, 'F':3}  

# Diskretizavimo dažnis:
# fs = 200

# /////////////////////////////////////////////////////////////////

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) bei kitais sąrašais", rec_dir)
print("Failas su EKG įrašų SubjCode train sąrašu ", train_subjcode_lst_fname)
print("Failas su EKG įrašų SubjCode validation sąrašu ", validation_subjcode_lst_fname)
print("Failas su EKG įrašų SubjCode test sąrašu ", test_subjcode_lst_fname)

print("\nAplankas su apskaičiuotais požymių masyvais ", features_dir)
print("Train failas su apskaičiuotais požymiais ", train_feature_set_fname)
print("Validation failas su apskaičiuotais požymiais ", validation_feature_set_fname)
print("Test failas su apskaičiuotais požymiais ", test_feature_set_fname)

print("\nTrain failas su praleistais požymiais ", train_omitted_set_fname)
print("Validation failas su praleistais požymiais ", validation_omitted_set_fname)
print("Test failas su apskaičiuotais požymiais ", test_omitted_set_fname)

# print("Pūpsnių atributų failas:", all_beats_attr_fname) # Atsisakome, formuojame beskaičiuodami
# kiekvienam įrašui atskrai
# print("Diskretizavimo dažnis: ", fs)
# print('Klasifikavimo schema:', selected_beats)
# print('Klasių skaičius:', len(selected_beats))

print("\n")

In [ ]:
# Pagrindinis skriptas - sukūriamas diske csv failas su EKG įrašų pūpsnių požymiais
# bei pūpsnių atributais: SubjCode, pūpsnio indeksas, pūpsnio anotacija.
# Einama per ECG įrašus, apskaičiuojamas kiekvienam įrašui pūpsnių požymių datafreimas,
# suradus požymius, csv failas diske papildomas apskaičiuotais požymiais ir atributais.
# Taip pat suformuojami ir įrašomi į csv failus omitted masyvai (failai su praleistais požymiais). 

def get_beat_features_set(rec_dir, SubjCodes, features_filepath, selected_features, omitted_filepath):

    # Ciklas per pacientų įrašus
    for SubjCode in SubjCodes:
        
        # Nuskaitome EKG įrašą (npy formatu)
        sign_raw = read_rec(rec_dir, SubjCode)
        signal_length = sign_raw.shape[0]
        signal = sign_raw

        # Surandame ir išvedame įrašo atributus
        file_name = get_filename(rec_dir, SubjCode)
        userNr, recNr = split_SubjCode(SubjCode)
        print(f"SubjCode: {SubjCode} userNr: {userNr:>2} file_name: {file_name:>2} signal_length: {signal_length}")

        # Filtruojame signalą
        # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

        # Nuskaitome paciento anotacijas ir jų indeksus
        df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
        atr_sample = df_rpeaks['sampleIndex'].to_numpy()
        # atr_symbol = df_rpeaks['annotationValue'].to_numpy()

        idx_lst = list(range(1, len(atr_sample)-1))
        # idx_lst = [1,2,3] # Derinimui susimažiname pūpsnių skaičių

        # Formuojame iš pūpsnių požymių datafreimą
        data_frame, omitted = get_beat_features_set_fda_vu_v1(signal, df_rpeaks, idx_lst)

# Sutvarkome ir įrašome į failą požymių masyvus

        # Sutvarkome indeksus ir stulpelių pavadinimus
        data_frame.reset_index(inplace=True)
        data_frame.columns = data_frame.columns.astype(str)

        # Derinimui susimažiname apimtis
        data_frame = data_frame[selected_features] # Derinimui sumažiname požymių kiekį
        # data_frame = data_frame.drop(data_frame.index[20:]) # Derinimui sumažiname eilučių skaičių

        # Pridedame prie požymių jungtinį paciento ir įrašo atributą SubjCode
        data_frame = data_frame.assign(SubjCode=lambda x: SubjCode)
        col = data_frame.pop("SubjCode")
        data_frame.insert(0, col.name, col)
        # print(data_frame.head(20))

        # Įrašome požymių datafreimą į csv failą
        file = Path(features_filepath)
        if file.exists():
            # print ("\nFile exist")
            data_frame.to_csv(features_filepath, mode='a', index=False, header=False)
        else:
            # print ("\nFile not exist")
            data_frame.to_csv(features_filepath, index=False, header=True)
        # print(f'\nPožymių masyvas įrašytas į:  {features_filepath}')

# Sutvarkome ir įrašome į failą omitted masyvus
        if (omitted.empty is not True):
            # print('\nomitted:', list(omitted["idx"]))
            # print(omitted.head(20))

# Sutvarkome indeksus 
            omitted.reset_index(inplace=True)
            omitted.drop('index', inplace=True, axis=1)

            # Pridedame prie požymių jungtinį paciento ir įrašo atributą SubjCode
            omitted = omitted.assign(SubjCode=lambda x: SubjCode)
            col = omitted.pop("SubjCode")
            omitted.insert(0, col.name, col)
            # print(omitted.head(20))

            # Įrašome omitted datafreimą į csv failą
            file = Path(omitted_filepath)
            if file.exists():
                # print ("\nFile exist")
                omitted.to_csv(omitted_filepath, mode='a', index=False, header=False)
            else:
                # print ("\nFile not exist")
                omitted.to_csv(omitted_filepath, index=False, header=True)
            # print(f'\nOmitted masyvas įrašytas į:  {omitted_filepath}')


    

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
# warnings.filterwarnings("ignore")

print("\nAtliekama pūpsnių pacientų įrašuose požymių skaičiavimas")

# Testavimui sumažiname požymių skaičių:
selected_features = ['idx', 'seq_size', 'label', 'RRl', 'RRr', 'RRl/RRr', 'wl_side', 'wr_side',
'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val', 'P_pos', 'Q_pos', 'R_pos',
 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT']

# Kas kiek išvedamas apdorotų sekų skaičius
# show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
# class_names = list(selected_beats.keys()) 
# n_classes = len(selected_beats)
# print(class_names)

# Sukuriame aplanką požymių failams
create_dir(features_dir)

# TOLIAU FORMUOJAME POŽYMIŲ IR OMITTED MASYVUS IR ĮRAŠOME Į FAILUS:
# - train_feature_set_fname, train_omitted_set_fname
# - validation_feature_set_fname, validation_omitted_set_fname
# - test_feature_set_fname, test_omitted_set_fname


In [14]:
# ------------------------- Variantas: mokymo imtis  
print('\nMokymo imtis')

# Pacientų įrašų sąrašas testavimui
file_path = Path(rec_dir, 'train_subjcode_lst.csv')
# train_SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
train_SubjCodes = [10020,10021] #Testavimui

print("Train įrašų sąrašas:", train_SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Panaikiname prieš tai buvusius požymių failus
features_filepath = Path(features_dir, train_feature_set_fname)
path = Path(features_filepath)
if path.exists():
    path.unlink()
# path.unlink(missing_ok=True)  - gali būti naudojamas tik su python 3.8

omitted_filepath = Path(features_dir, train_omitted_set_fname)
path = Path(omitted_filepath)
if path.exists():
    path.unlink()

# Požymių datafreimų formavimas
start_time = time.time()
get_beat_features_set(rec_dir, train_SubjCodes, features_filepath, selected_features, omitted_filepath)
end_time = time.time()
runtime(end_time-start_time)

# Eilučių skaičius csv masyve
path = Path(features_filepath)
if path.exists():
    number_of_rows = count_lines_enumrate(features_filepath)
    print(f"\nSuformuotas požymių masyvas train. Eilučių skaičius masyve: {number_of_rows}")
    print(f'Požymių masyvas įrašytas į:  {features_filepath}')
else:
    print('\nPožymių masyvas nesuformuotas')    

# Eilučių skaičius omitted masyve
path = Path(omitted_filepath)
if path.exists():
    number_of_rows = count_lines_enumrate(omitted_filepath)
    print(f"\nSuformuotas masyvas omitted. Eilučių skaičius masyve: {number_of_rows}")
    print(f'Omitted masyvas įrašytas į:  {omitted_filepath}')
else:
    print('\nOmitted masyvas nesuformuotas')    

# Bandomasis požymių datafreimų skaitymas iš csv failo
# print(f'\nBandomasis skaitymas. Požymių masyvas iš:  {filepath}')
# filepath = Path(features_dir, train_feature_set_fname)
# data_frame = pd.read_csv(filepath)
# print(data_frame.head(6))



Mokymo imtis
Train įrašų sąrašas: [10020, 10021]
Sąrašas nuskaitytas iš: /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy/train_subjcode_lst.csv
SubjCode: 10020 userNr: 1002 file_name: 1625408.182 signal_length: 127999
SubjCode: 10021 userNr: 1002 file_name: 1625402.027 signal_length: 127999
Runtime: 00:00:19

Suformuotas požymių masyvas train. Eilučių skaičius masyve: 1935
Požymių masyvas įrašytas į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/sets/train_features_tst.csv

Suformuotas masyvas omitted_. Eilučių skaičius masyve: 11
Omitted masyvas įrašytas į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/sets/train_omitted_tst.csv


In [15]:
# --------------------------- Variantas: validation imtis  
print('\nValidation imtis')
file_path = Path(rec_dir, 'validation_subjcode_lst.csv')
# validation_SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
validation_SubjCodes = [10011, 10012] #Testavimui

print("validation įrašų sąrašas:", validation_SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Panaikiname prieš tai buvusius požymių failus
features_filepath = Path(features_dir, validation_feature_set_fname)
path = Path(features_filepath)
if path.exists():
    path.unlink()
# path.unlink(missing_ok=True)  - gali būti naudojamas tik su python 3.8

omitted_filepath = Path(features_dir, validation_omitted_set_fname)
path = Path(omitted_filepath)
if path.exists():
    path.unlink()

# Požymių datafreimų formavimas
start_time = time.time()
get_beat_features_set(rec_dir, validation_SubjCodes, features_filepath, selected_features, omitted_filepath)
end_time = time.time()
runtime(end_time-start_time)

# Eilučių skaičius csv masyve
path = Path(features_filepath)
if path.exists():
    number_of_rows = count_lines_enumrate(features_filepath)
    print(f"\nSuformuotas požymių masyvas train. Eilučių skaičius masyve: {number_of_rows}")
    print(f'Požymių masyvas įrašytas į:  {features_filepath}')
else:
    print('\nPožymių masyvas nesuformuotas')    

# Eilučių skaičius omitted masyve
path = Path(omitted_filepath)
if path.exists():
    number_of_rows = count_lines_enumrate(omitted_filepath)
    print(f"\nSuformuotas masyvas omitted. Eilučių skaičius masyve: {number_of_rows}")
    print(f'Omitted masyvas įrašytas į:  {omitted_filepath}')
else:
    print('\nOmitted masyvas nesuformuotas') 
       
# Bandomasis požymių datafreimų skaitymas iš csv failo
# print(f'\nBandomasis skaitymas. Požymių masyvas iš:  {filepath}')
# filepath = Path(features_dir, validation_feature_set_fname)
# data_frame = pd.read_csv(filepath)
# print(data_frame.head(3))


Validation imtis
validation įrašų sąrašas: [10011, 10012]
Sąrašas nuskaitytas iš: /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy/validation_subjcode_lst.csv
SubjCode: 10011 userNr: 1001 file_name: 1626330.744 signal_length: 127999
SubjCode: 10012 userNr: 1001 file_name: 1626924.927 signal_length: 127999
Runtime: 00:00:17

Suformuotas požymių masyvas train. Eilučių skaičius masyve: 1537
Požymių masyvas įrašytas į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/sets/validation_features_tst.csv

Suformuotas masyvas omitted_. Eilučių skaičius masyve: 2
Omitted masyvas įrašytas į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/sets/validation_omitted_tst.csv


In [16]:
# --------------------------------------------------- Variantas: testinė imtis  
print('\nTest imtis')
file_path = Path(rec_dir, 'test_subjcode_lst.csv')
# test_SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))
test_SubjCodes = [10000, 10001] #Testavimui

print("test įrašų sąrašas:", test_SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Panaikiname prieš tai buvusius požymių failus
features_filepath = Path(features_dir, test_feature_set_fname)
path = Path(features_filepath)
if path.exists():
    path.unlink()
# path.unlink(missing_ok=True)  - gali būti naudojamas tik su python 3.8

omitted_filepath = Path(features_dir, test_omitted_set_fname)
path = Path(omitted_filepath)
if path.exists():
    path.unlink()

# Požymių datafreimų formavimas
start_time = time.time()
get_beat_features_set(rec_dir, test_SubjCodes, features_filepath, selected_features, omitted_filepath)
end_time = time.time()
runtime(end_time-start_time)

# Eilučių skaičius csv masyve
path = Path(features_filepath)
if path.exists():
    number_of_rows = count_lines_enumrate(features_filepath)
    print(f"\nSuformuotas požymių masyvas train. Eilučių skaičius masyve: {number_of_rows}")
    print(f'Požymių masyvas įrašytas į:  {features_filepath}')
else:
    print('\nPožymių masyvas nesuformuotas')    

# Eilučių skaičius omitted masyve
path = Path(omitted_filepath)
if path.exists():
    number_of_rows = count_lines_enumrate(omitted_filepath)
    print(f"\nSuformuotas masyvas omitted. Eilučių skaičius masyve: {number_of_rows}")
    print(f'Omitted masyvas įrašytas į:  {omitted_filepath}')
else:
    print('\nOmitted masyvas nesuformuotas')    

# Bandomasis požymių datafreimų skaitymas iš csv failo
# print(f'\nBandomasis skaitymas. Požymių masyvas iš:  {filepath}')
# filepath = Path(features_dir, test_feature_set_fname)
# data_frame = pd.read_csv(filepath)
# print(data_frame.head(3))


# --------------------------------------------------- Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
# file_path = Path(rec_dir,'info_create_z.json')
# with open(file_path) as json_file:
#     info_create = json.load(json_file)
# SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas



Test imtis
test įrašų sąrašas: [10000, 10001]
Sąrašas nuskaitytas iš: /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy/test_subjcode_lst.csv
SubjCode: 10000 userNr: 1000 file_name: 1630673.825 signal_length: 127999
SubjCode: 10001 userNr: 1000 file_name: 1626941.468 signal_length: 127999
Runtime: 00:00:17

Suformuotas požymių masyvas train. Eilučių skaičius masyve: 1622
Požymių masyvas įrašytas į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/sets/test_features_tst.csv

Suformuotas masyvas omitted_. Eilučių skaičius masyve: 1
Omitted masyvas įrašytas į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/sets/test_omitted_tst.csv
